### Other fragments to test
- pencil: pensées détachées
- ink: mélanges
- fr-sauvage
- fr on russia and china
- fr selected fragments 51-55-72-40

In [1]:
import pandas as pd
import numpy as np

import re

from sklearn.preprocessing import StandardScaler

import logging

logging.basicConfig(level="INFO")
logger = logging.getLogger("ruzicka")

from ruzicka.BDIVerifier import BDIVerifier

In [39]:
# load preprocessed data

corpus = pd.read_csv('05_tests/rchina_2k_200mfw_rfreq.csv') # rel freq

corpus

,work,author,chunk_num,tag,de,des,et,la,les,vous,...,voit,rendre,beaucoup,seul,ici,mieux,donner,enfin,voir,cependant
0,Avis,Baudeau,0,0__Baudeau_Avis,3.45,2.20,2.50,1.40,2.35,0.65,...,0.05,0.00,0.05,0.05,0.05,0.00,0.05,0.20,0.00,0.00
1,Eclaircissemens,Baudeau,0,0__Baudeau_Eclaircissemens,4.40,2.85,2.80,2.60,2.40,1.65,...,0.05,0.00,0.00,0.05,0.00,0.00,0.05,0.00,0.00,0.00
2,Explication,Baudeau,0,0__Baudeau_Explication,4.90,2.60,3.80,3.10,4.65,0.95,...,0.00,0.00,0.05,0.10,0.10,0.00,0.05,0.05,0.05,0.00
3,Idees sur l-administration,Baudeau,0,0__Baudeau_Idees sur l-administration,5.60,1.90,2.20,3.65,1.85,0.10,...,0.00,0.00,0.10,0.10,0.00,0.05,0.05,0.10,0.00,0.05
4,Idees sur la puissance,Baudeau,0,0__Baudeau_Idees sur la puissance,4.90,1.85,2.50,2.35,2.75,0.10,...,0.00,0.10,0.05,0.00,0.00,0.10,0.00,0.10,0.05,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4421,elements T3,Marmontel,96,96__Marmontel_elements T3,4.15,1.20,3.35,3.00,1.15,0.00,...,0.05,0.10,0.00,0.25,0.00,0.05,0.05,0.05,0.00,0.00
4422,Histoire Generale T19,Deleyre,97,97__Deleyre_Histoire Generale T19,5.25,1.85,0.75,3.75,2.65,0.00,...,0.00,0.05,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4423,elements T3,Marmontel,97,97__Marmontel_elements T3,4.60,0.95,3.10,3.40,1.55,0.00,...,0.00,0.10,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00
4424,Histoire Generale T19,Deleyre,98,98__Deleyre_Histoire Generale T19,4.90,2.95,0.75,3.25,3.20,0.05,...,0.05,0.00,0.20,0.00,0.00,0.05,0.00,0.00,0.05,0.05


In [40]:
raw_df = corpus.iloc[:, :4] # leaves metadata
X = corpus.iloc[:, 4:] # leaves only word columns

raw_df

,work,author,chunk_num,tag
0,Avis,Baudeau,0,0__Baudeau_Avis
1,Eclaircissemens,Baudeau,0,0__Baudeau_Eclaircissemens
2,Explication,Baudeau,0,0__Baudeau_Explication
3,Idees sur l-administration,Baudeau,0,0__Baudeau_Idees sur l-administration
4,Idees sur la puissance,Baudeau,0,0__Baudeau_Idees sur la puissance
...,...,...,...,...
4421,elements T3,Marmontel,96,96__Marmontel_elements T3
4422,Histoire Generale T19,Deleyre,97,97__Deleyre_Histoire Generale T19
4423,elements T3,Marmontel,97,97__Marmontel_elements T3
4424,Histoire Generale T19,Deleyre,98,98__Deleyre_Histoire Generale T19


In [41]:
# numer of rows and columns in the X (word frequencies subset)
X.shape

(4426, 200)

In [42]:
# create numeric author labels
labels, label_uniques = raw_df.author.factorize()
raw_df.insert(1, "author_label", labels)
raw_df

,work,author_label,author,chunk_num,tag
0,Avis,0,Baudeau,0,0__Baudeau_Avis
1,Eclaircissemens,0,Baudeau,0,0__Baudeau_Eclaircissemens
2,Explication,0,Baudeau,0,0__Baudeau_Explication
3,Idees sur l-administration,0,Baudeau,0,0__Baudeau_Idees sur l-administration
4,Idees sur la puissance,0,Baudeau,0,0__Baudeau_Idees sur la puissance
...,...,...,...,...,...
4421,elements T3,11,Marmontel,96,96__Marmontel_elements T3
4422,Histoire Generale T19,3,Deleyre,97,97__Deleyre_Histoire Generale T19
4423,elements T3,11,Marmontel,97,97__Marmontel_elements T3
4424,Histoire Generale T19,3,Deleyre,98,98__Deleyre_Histoire Generale T19


#### NB CHANGE PROBLEM 

In [43]:
# "pencil-pensees-detachees", "ink-melanges",
# "fr-51-55-72-40","fr-on-russia-china", "fr-sauvage"

problems = raw_df[raw_df.work == "fr-on-russia-china"].reset_index(drop=True).copy()
problems_X = X[raw_df.work == "fr-on-russia-china"].reset_index(drop=True).copy()
problems

,work,author_label,author,chunk_num,tag
0,fr-on-russia-china,7,HDI,0,0__HDI_fr-on-russia-china
1,fr-on-russia-china,7,HDI,1,1__HDI_fr-on-russia-china


In [44]:
# extract the rest of the corpus

rest = raw_df[raw_df.work != "fr-on-russia-china"].reset_index(drop=True).copy()
rest_X = X[raw_df.work != "fr-on-russia-china"].reset_index(drop = True).copy()
rest

,work,author_label,author,chunk_num,tag
0,Avis,0,Baudeau,0,0__Baudeau_Avis
1,Eclaircissemens,0,Baudeau,0,0__Baudeau_Eclaircissemens
2,Explication,0,Baudeau,0,0__Baudeau_Explication
3,Idees sur l-administration,0,Baudeau,0,0__Baudeau_Idees sur l-administration
4,Idees sur la puissance,0,Baudeau,0,0__Baudeau_Idees sur la puissance
...,...,...,...,...,...
4419,elements T3,11,Marmontel,96,96__Marmontel_elements T3
4420,Histoire Generale T19,3,Deleyre,97,97__Deleyre_Histoire Generale T19
4421,elements T3,11,Marmontel,97,97__Marmontel_elements T3
4422,Histoire Generale T19,3,Deleyre,98,98__Deleyre_Histoire Generale T19


**Scaling**

In [45]:
sts = StandardScaler(with_mean=False).fit(rest_X)

rest_scaled_X = sts.transform(rest_X)
problems_scaled_X = sts.transform(problems_X)

**Verification**

In [46]:
# check unique authors
label_uniques.values

array(['Baudeau', 'Chastellux', 'Condorcet', 'Deleyre', 'dHolbach',
       'Diderot', 'Guibert', 'HDI', 'Jaucourt', 'Jussieu', 'La Grange',
       'Marmontel', 'Meister', 'Morellet', 'Naigeon', 'Pechmeja',
       'Raynal', 'Rivière', 'Saint-Lambert'], dtype=object)

In [47]:
# set verifier
rng = np.random.default_rng(42)

bdi_mm = BDIVerifier(
    metric='minmax', nb_bootstrap_iter=1000, rnd_prop=0.35, random_state=rng
)

In [48]:
# fit 
bdi_mm.fit(rest_scaled_X, rest.author_label)

01/22/2025 06:40:27 [ruzicka:INFO] Fitting on 4424 documents...


In [49]:
for label in label_uniques.values:
    print(f"Testing against {label}")
    code = label_uniques.get_loc(label)
    print(
        f"Bootstrap Match Strength (one per chunk, 0-1.0): {bdi_mm.predict_proba(problems_scaled_X, [code] * problems_scaled_X.shape[0])}"
    )

01/22/2025 06:40:28 [ruzicka:INFO] Predicting on 2 documents


Testing against Baudeau


01/22/2025 06:40:29 [ruzicka:INFO] Predicting on 2 documents


Bootstrap Match Strength (one per chunk, 0-1.0): [0.709 0.901]
Testing against Chastellux


01/22/2025 06:40:29 [ruzicka:INFO] Predicting on 2 documents


...

Bootstrap Match Strength (one per chunk, 0-1.0): [0.085 0.383]
Testing against Raynal


01/22/2025 06:40:38 [ruzicka:INFO] Predicting on 2 documents


Bootstrap Match Strength (one per chunk, 0-1.0): [0.941 0.927]
Testing against Rivière


01/22/2025 06:40:38 [ruzicka:INFO] Predicting on 2 documents


Bootstrap Match Strength (one per chunk, 0-1.0): [0.479 0.528]
Testing against Saint-Lambert
Bootstrap Match Strength (one per chunk, 0-1.0): [0.862 0.961]


#### NB CHANGE PATH

In [50]:
authors = label_uniques.values

fh = ''

for a in authors:

    print(a)
    
    bdi_mm.predict_proba(
        problems_scaled_X, [label_uniques.get_loc(a)] * problems_scaled_X.shape[0]
    )

    
    x = pd.DataFrame(dict(zip(problems.tag, bdi_mm._dist_arrays)))
    
    fh = '05_tests/tests/rchina_vs_' + a + '.csv'

    x.to_csv(fh)

01/22/2025 06:40:46 [ruzicka:INFO] Predicting on 2 documents


Baudeau


01/22/2025 06:40:47 [ruzicka:INFO] Predicting on 2 documents


Chastellux


01/22/2025 06:40:47 [ruzicka:INFO] Predicting on 2 documents


...

Raynal


01/22/2025 06:40:56 [ruzicka:INFO] Predicting on 2 documents


Rivière


01/22/2025 06:40:56 [ruzicka:INFO] Predicting on 2 documents


Saint-Lambert
